<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_korEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'WordNet만을 단순히 바꿔서 결괏값을 내기 때문에 의미가 변형되어버리는 경우가 생깁니다. 특히 SR과 RI를 사용할 때 많이 발생하는데 제가 잘못한 건 아닌 것 같아요 를 제가 잘못한 총 아닌 것 같아요 (건 -> 총) 으로 바뀌기도 한다. 본 논문에서는 이렇게 바꿔도 꽤나 원문 데이터의 성질을 따라간다고 하지만.. 한국어의 특성상 완전히 따라가기에는 쉽지 않은 것 같다.

안전하게 데이터 증강을 하고 싶다면 RD, RS만을 사용하고, 데이터가 많이 필요하다싶으면 SR과 RI까지 사용하고 인간지능으로 데이터를 걸러내는 작업이 필요할 것이다.'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re
import pickle

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

# 데이터csv 파일 불러오기

In [ ]:
df_train = pd.read_csv(f"{DATA_PATH}0906.csv")
df_train.head()

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score,kiwi_reviews,keybert_reviews
0,비 오는데도 배달 빨리 해주시고 감사합니다! 맛있어요옹,"안녕하세요 마라하오죽전점 입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해주셔서 ...",5,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,"안녕하세요 마라 하오 죽전점입니다. 즐거운 시간 보내시고, 소중한 리뷰 작성해 주셔...",마라 하오,죽전점,마라,비 오는데도 배달 빨리해주시고 감사합니다! 맛있어요,1,0.761931,"['배달', '감사', '맛있']","[('배달', 0.8359), ('감사', 0.8353), ('맛있', 0.4462)]"
1,2인세트인데 양도 엄청많아서 좋았어요! 맛있게 잘 먹었습니당,놀부부대찌개를 이용해주셔서 감사합니다 맛있는 음식을 넉넉하게 드실수 있도록 메뉴구성...,4,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,놀부부대찌개를 이용해 주셔서 감사합니다 맛있는 음식을 넉넉하게 드실 수 있도록 메뉴...,"놀부,부대찌,대찌",,부대찌개,2인 세트인데 양도 엄청 많아서 좋았어요! 맛있게 잘 먹었습니다,1,0.586811,"['세트', '많', '좋', '맛있']","[('세트', 0.806), ('맛있', 0.5224)]"
2,너무맛있어요! 자주시켜먹게되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디져트와 음료...,5,너무 맛있어요! 자주 시켜 먹게 되네요,은근히 당기는 맛들로 준비하고 있습니다 직원들의 입맛을 통과한 맛있는 디저트와 음료...,NaN,,,너무 맛있어요! 자주 시켜 먹게 되네요,1,0.779962,['맛있'],"[('맛있', 1.0)]"
3,항상 맛있어요 서비스도쵝오,주문 감사드립니다 맛있게 드시고요 환절기에 감기조심하시고요 앞으로도 만족을 드릴수 ...,5,항상 맛있어요 서비스도 최고,주문 감사드립니다 맛있게 드시고요 환절기에 감기 조심하시고요 앞으로도 만족을 드릴 ...,NaN,,,항상 맛있어요 서비스도 최고,1,0.801292,"['맛있', '서비스', '최고']","[('서비스', 0.9012), ('최고', 0.5062), ('맛있', 0.5062)]"
4,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 화이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,5,모든 반찬이 다 맛있습니다!,주문 감사드립니다 이번 주 월요일도 파이팅입니다~ 미세먼지 조심하시고 감기도 조심하세요~,NaN,,,모든 반찬이 다 맛있습니다!,1,0.762959,"['반찬', '맛있']","[('반찬', 0.9452), ('맛있', 0.4905)]"


In [ ]:
for i in range(0,2):
    num_rating = df_train[df_train['sentiment'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / df_train.shape[0] * 100}%")

별점 0점 개수: 1318, 6.3597761049990345%
별점 1점 개수: 19406, 93.64022389500096%


In [ ]:
t = df_train['corrected_review'][9]
print(t)
len(t)

맛있게 잘 먹었습니다 담에는 한 마리씩 시켜야 맛대로 시켜야겠어요 양이 적어요


43

In [ ]:
text = df_train['corrected_review'][203]
text

'다른 지점에서 육회비빔밥 시켜 먹었을 때 진짜 먹어본 육회비빔밥 중에서 최악이었는데 여기 지점은 맛있네요 앞으로도 육회비빔밥 당길 때면 여기서 시킬 거 같네요 잘 먹었습니다'

In [ ]:
len(text)

96

# EDA

In [ ]:
# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('[^ ㄱ-ㅣ가-힣]+').sub('',line)
	return parseText

In [ ]:
########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

In [ ]:
########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

In [ ]:
def EDA(sentence, alpha_rs=0.1, p_rd=0.1, num_aug=3):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	# words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_rs = max(1, int(alpha_rs*num_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

In [ ]:
EDA(text)

['다른 지점에서 육회비빔밥 시켜 먹었을 때 진짜 먹어본 거 중에서 때면 여기 지점은 맛있네요 앞으로도 육회비빔밥 당길 최악이었는데 여기서 시킬 육회비빔밥 같네요 잘 먹었습니다',
 '다른 지점에서 육회비빔밥 먹었을 때 진짜 먹어본 육회비빔밥 중에서 최악이었는데 여기 지점은 앞으로도 육회비빔밥 당길 때면 여기서 거 같네요 잘 먹었습니다',
 '다른 지점에서 육회비빔밥 시켜 먹었을 때 진짜 먹어본 육회비빔밥 중에서 최악이었는데 여기 지점은 맛있네요 앞으로도 육회비빔밥 당길 때면 여기서 시킬 거 같네요 잘 먹었습니다']

# 부정리뷰

In [ ]:
ng_df = df_train.loc[df_train.sentiment == 0]
ng_df

,고객리뷰,사장댓글,별점,corrected_review,corrected_reply,store,branch,food,review,sentiment,score,kiwi_reviews,keybert_reviews
19,"야채,단무지 넣지 말라고 요청 드렸는데 주먹밥에 들어거는 김까지 안보내셨네요 담엔 ...",먼저 죄송 합니다 바로 전화 주셨으면 김가루 가져다드릴건데 죄송 합니다 재주문시 꼭...,4,"야채, 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔...",먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,NaN,,야채,"야채, 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔...",0,0.817330,"['야채', '단무지', '요청', '주먹밥']","[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63..."
80,아파서 시켰어요 저는 참깨김후레이크? 그거가 저랑 잘 안맞네요.,안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,4,아파서 시켰어요 저는 참깨 김 플레이크? 그거가 저랑 잘 안 맞네요.,안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,NaN,,,아파서 시켰어요 저는 참깨 김 플레이크? 그거가 저랑 잘 안 맞네요.,0,0.782832,"['아프', '참깨', '레이크', '그거']","[('레이크', 0.7232), ('그거', 0.6273), ('아프', 0.626..."
86,예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십년만에 주문을 해봅니다. 양 넉...,안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,4,예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십 년 만에 주문을 해봅니다. 양...,안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,NaN,,,예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십 년 만에 주문을 해봅니다. 양...,0,0.335445,"['예전', '만다린', '주문', '수십', '주문', '국물', '대왕오징어']","[('대왕오징어', 0.7619), ('만다린', 0.5751), ('수십', 0...."
95,국은 시원하게 맛있었어요 국물양이 더 많았으면 더 좋았겠습니다,맛있게 드셨다니 감사드립니다. 매장과 같이 최대한 드리고있습니다. 배달용기에 국물을...,4,국은 시원하게 맛있었어요 국물 양이 더 많았으면 더 좋았겠습니다,맛있게 드셨다니 감사드립니다. 매장과 같이 최대한 드리고 있습니다. 배달용기에 국물...,NaN,,,국은 시원하게 맛있었어요 국물 양이 더 많았으면 더 좋았겠습니다,0,0.478211,"['시원하', '맛있', '국물', '많', '좋']","[('시원하', 0.7902), ('국물', 0.7239), ('맛있', 0.4535)]"
111,대통령 각하 만세.,주문 감사합니다~추운 날씨 감기조심하세요~,5,대통령 각하 만세.,주문 감사합니다~추운 날씨 감기 조심하세요~,NaN,,,대통령 각하 만세.,0,0.371106,"['대통령', '각하']","[('대통령', 0.8653), ('각하', 0.6785)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20711,여름엔 아무래도 좀 바삭한 맛이 있어야 하는 것 같아요 카페에서 파는 와플이 그렇게...,더 맛있는 요리로 찾아 뵐게요 이번 여름엔 더 맛있게 어떤 요리든 만들어 보겠습니다.,4,여름엔 아무래도 좀 바삭한 맛이 있어야 하는 것 같아요 카페에서 파는 와플이 그렇게...,더 맛있는 요리로 찾아뵐게요 이번 여름엔 더 맛있게 어떤 요리든 만들어 보겠습니다.,NaN,,,여름엔 아무래도 좀 바삭한 맛이 있어야 하는 것 같아요 카페에서 파는 와플이 그렇게...,0,0.562077,"['여름', '같', '카페', '와플', '그렇']","[('카페', 0.739), ('와플', 0.7204), ('여름', 0.5952)..."
20712,붕어빵은 냉동이 아닌 생것으로 먹는걸 선호해서 자주 사 먹는데 오늘은 유난히 부친 ...,항상 친절한 매장 운영을 위해 노력하겠습니다.,4,붕어빵은 냉동이 아닌 생것으로 먹는 걸 선호해서 자주 사 먹는데 오늘은 유난히 부친...,항상 친절한 매장 운영을 위해 노력하겠습니다.,NaN,,,붕어빵은 냉동이 아닌 생것으로 먹는 걸 선호해서 자주 사 먹는데 오늘은 유난히 부친...,0,0.697954,"['붕어빵', '냉동', '선호', '오늘', '붕어빵', '아쉽']","[('냉동', 0.7186), ('오늘', 0.6781), ('선호', 0.5995..."
20716,우리나라에서 나는 어묵처럼 생긴 것을 어묵이라고 생각하면 거부감이 들 수도 있을 것...,그래도 예쁘게 잘 올려주신 사진 덕분에 제가 마음이 놓였습니다 그럼 이만 사진을 봐...,4,우리나라에서 나는 어묵처럼 생긴 것을 어묵이라고 생각하면 거부감이 들 수도 있을 것...,그래도 예쁘게 잘 올려주신 사진 덕분에 제가 마음이 놓였습니다 그럼 이만 사진을 봐...,NaN,,,우리나라에서 나는 어묵처럼 생긴 것을 어묵이라고 생각하면 거부감이 들 수도 있을 것...,0,0.352121,"['우리', '나라', '어묵', '생각', '거부감', '있', '같']","[('거부감', 0.7177), ('생각', 0.6954), ('어묵', 0.655..."
20717,맛은 정말 괜찮은데 기본으로 주는 쌀국수가 양이 너무 적어요 곱배기에 계란 추가한건...,오늘 부족했던 부분 많이 보충해주셨는데 또 궁금하신 부분이 있으면 언제든 질문해주시...,4,맛은 정말 괜찮은데 기본으로 주는 쌀국수가 양이 너무 적어요 곱빼기에 계란 추가한 ...,오늘 부족했던 부분 많이 보충해 주셨는데 또 궁금하신 부분이 있으면 언제든 질문해 ...,NaN,,,맛은 정말 괜찮은데 기본으로 주는 쌀국수가 양이 너무 적어요 곱빼기에 계란 추가한 ...,0,0.649096,"['괜찮', '기본', '쌀국수', '적', '곱빼기', '계란', '추가', '그...","[('배부르', 0.6836), ('쌀국수', 0.628), ('기본', 0.579..."


- 길이가 짧지 않은 리뷰에 대해서만 증강 적용

In [ ]:
for index, row in ng_df.iterrows():
    print(row)
    break

고객리뷰                야채,단무지 넣지 말라고 요청 드렸는데 주먹밥에 들어거는 김까지 안보내셨네요 담엔 ...
사장댓글                먼저 죄송 합니다 바로 전화 주셨으면 김가루 가져다드릴건데 죄송 합니다 재주문시 꼭...
별점                                                                  4
corrected_review    야채, 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔...
corrected_reply     먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...
store                                                             NaN
branch                                                               
food                                                               야채
review              야채, 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔...
sentiment                                                           0
score                                                         0.81733
kiwi_reviews                               ['야채', '단무지', '요청', '주먹밥']
keybert_reviews     [('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...
Name: 19, dtype: object


In [ ]:
augmented_reviews = [] # 리뷰 데이터에 대해서만 EDA 증강
augmented_replies = []
keybert_reviews = []
store = []
branch = []
food = []
sentiment = []

for index, row in ng_df.iterrows():
    if len(row['corrected_review']) > 50:
        augmented = EDA(row["corrected_review"])
        augmented_reviews.extend(augmented)
        augmented_replies.extend([row["corrected_reply"]] * len(augmented))
        keybert_reviews.extend([row['keybert_reviews']] * len(augmented))
        store.extend([row["store"]] * len(augmented))
        branch.extend([row["branch"]] * len(augmented))
        food.extend([row["food"]] * len(augmented))
        sentiment.extend([row["sentiment"]] * len(augmented))
    else:
        augmented_reviews.append(row["corrected_review"])
        augmented_replies.append(row["corrected_reply"])
        keybert_reviews.append(row['keybert_reviews'])
        store.append(row["store"])
        branch.append(row["branch"])
        food.append(row["food"])
        sentiment.append(row["sentiment"])

In [ ]:
df_augmented = pd.DataFrame({'고객리뷰': augmented_reviews,
                             '사장댓글': augmented_replies,
                             'keybert_reviews': keybert_reviews,
                             'store': store,
                             'branch': branch,
                             'food': food,
                             'sentiment': sentiment
                             })

df_augmented

,고객리뷰,사장댓글,keybert_reviews,store,branch,food,sentiment
0,야채 단무지 넣지 말라고 안 주먹밥에 들어거는 김까지 요청드렸는데 보내셨네요 담엔 ...,먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",NaN,,야채,0
1,야채 단무지 넣지 말라고 요청드렸는데 들어거는 김까지 안 보내셨네요 담엔 잘 챙겨주세요,먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",NaN,,야채,0
2,야채 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 ...,먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",NaN,,야채,0
3,아파서 시켰어요 저는 참깨 김 플레이크? 그거가 저랑 잘 안 맞네요.,안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,"[('레이크', 0.7232), ('그거', 0.6273), ('아프', 0.626...",NaN,,,0
4,예전에는 해서 주문을 만다린에서만 먹었는데 정말 수십 년 만에 주문을 좀 양 넉넉했...,안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,"[('대왕오징어', 0.7619), ('만다린', 0.5751), ('수십', 0....",NaN,,,0
...,...,...,...,...,...,...,...
2719,정말 괜찮은데 기본으로 주는 쌀국수가 양이 너무 적어요 곱빼기에 계란 추가한 건데도...,오늘 부족했던 부분 많이 보충해 주셨는데 또 궁금하신 부분이 있으면 언제든 질문해 ...,"[('배부르', 0.6836), ('쌀국수', 0.628), ('기본', 0.579...",NaN,,,0
2720,맛은 정말 괜찮은데 기본으로 주는 쌀국수가 양이 너무 적어요 곱빼기에 계란 추가한 ...,오늘 부족했던 부분 많이 보충해 주셨는데 또 궁금하신 부분이 있으면 언제든 질문해 ...,"[('배부르', 0.6836), ('쌀국수', 0.628), ('기본', 0.579...",NaN,,,0
2721,시원한 국물이 국물 많았으면 더 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부으시면 국물이 흘러내리기 때문에 혹시라도 모자라면 댁에서 생...,"[('시원하', 0.7763), ('국물', 0.7322)]",NaN,,라면,0
2722,시원한 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 먹었으니 말이죠,배달용기에 국물을 더 부으시면 국물이 흘러내리기 때문에 혹시라도 모자라면 댁에서 생...,"[('시원하', 0.7763), ('국물', 0.7322)]",NaN,,라면,0


In [ ]:
df_augmented['store'] = df_augmented['store'].fillna("")
df_augmented.head()

,고객리뷰,사장댓글,keybert_reviews,store,branch,food,sentiment
0,야채 단무지 넣지 말라고 안 주먹밥에 들어거는 김까지 요청드렸는데 보내셨네요 담엔 ...,먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",,,야채,0
1,야채 단무지 넣지 말라고 요청드렸는데 들어거는 김까지 안 보내셨네요 담엔 잘 챙겨주세요,먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",,,야채,0
2,야채 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 ...,먼저 죄송합니다 바로 전화 주셨으면 김가루 가져다 드릴 건데 죄송합니다 재주문 시 ...,"[('주먹밥', 0.7787), ('요청', 0.7113), ('단무지', 0.63...",,,야채,0
3,아파서 시켰어요 저는 참깨 김 플레이크? 그거가 저랑 잘 안 맞네요.,안녕하세요! 찾아주셔서 감사합니다 그런데 고객님과 맞지 않는 부분으로 아쉬움이 남으...,"[('레이크', 0.7232), ('그거', 0.6273), ('아프', 0.626...",,,,0
4,예전에는 해서 주문을 만다린에서만 먹었는데 정말 수십 년 만에 주문을 좀 양 넉넉했...,안녕하세요 네 잘 알겠습니다 잘 살펴보겠습니다 감사합니다,"[('대왕오징어', 0.7619), ('만다린', 0.5751), ('수십', 0....",,,,0


In [ ]:
df_augmented[df_augmented['사장댓글'].str.contains('피자')].index

Int64Index([  83,  229,  230,  231,  237,  238,  239,  246,  295,  296,  297,
             324,  378,  384,  385,  386,  477,  478,  479,  480,  635,  726,
             755,  756,  757,  758,  759,  760,  817,  818,  819,  829,  833,
            1065, 1089, 1138, 1150, 1175, 1176, 1177, 1292, 1293, 1294, 1317,
            1527, 1581, 1582, 1583, 1589, 1590, 1591, 1606, 1613, 1614, 1615,
            1644, 1645, 1646, 1654, 1713, 1753, 1800, 1801, 1802, 1811, 1950,
            2086, 2087, 2088, 2202, 2203, 2204, 2545, 2603, 2604, 2605, 2606],
           dtype='int64')

In [ ]:
df_augmented['사장댓글'][83]

'일반 소스가 약간 매운맛이 있습니다 아마도 흑후추가 섞여있어서 그런 것 같습니다 언제나 노력하는 7번가 피자 용암점 되겠습니다 즐거운 시간 보내시고 행복한 나날 되세요 '

In [ ]:
df_augmented['store'][83]

'7번가 피자,7번,7번가'

In [ ]:
df_augmented = df_augmented.sample(frac=1).reset_index(drop=True) # 데이터 랜덤 섞기
df_augmented

,고객리뷰,사장댓글,keybert_reviews,store,branch,food,sentiment
0,"예전엔 반찬도 주셨던 것 같은데.,. 반찬도 없더라고요","놀부를 이용해 주셔서 감사합니다. 지금도 반찬은 나가고 있는데, 포장하면서 깜박했나...","[('반찬', 0.8752), ('예전', 0.4256)]",놀부,,,0
1,많은데 거의 없네요 주문했는데 바삭함이 없어 아쉬웠어요 스파게티는 맛있어요,안녕하세요! 토핑을 아끼지 않는 미. 친미 자입니다! 미. 친 이 피자 주문 주시고...,"[('스파게티', 0.7067), ('토핑', 0.5816), ('치즈', 0.44...",,,피자,0
2,오늘 번째 주문인데 오늘 것이 좀 떨어지네요 냉소바는 저번엔 간 딱 맞고 맛있어서 ...,헉! 소중한 의견 감사합니다~좀 더 세심히 살펴 만들어 드릴게요~부족한 식사가 되게...,"[('싱겁', 0.5641), ('저번', 0.3341), ('오늘', 0.2508...",,,,0
3,일단 맛은 있고요 냉면 치고 조금 비싸지만 배달하시는 분 전화 와서는 대뜸 안 들리...,"고객님, 안녕하세요 시간 내어 리뷰 작성해 주셔서 정말 감사합니다 항상 고객님들 주...","[('비싸', 0.5999), ('냉면', 0.5449), ('배달', 0.4959...","고기,갈비 민족",,갈비,0
4,최악입니다 맛은 무슨 신맛이 나고 덮밥이란 뜻을 모르시는 거 같은데 밥상 태가 진짜...,"고객님, 인생 제육을 주문해 주셔서 진심으로 감사합니다. 열심히 하겠습니다! 그리고...","[('며칠', 0.433), ('밥통', 0.4001), ('식감', 0.281),...",인생 제육,,볶음,0
...,...,...,...,...,...,...,...
2719,양은 많긴 한데 일단 막국수가 예전에 먹었던 그 기억이 있고 맛있었던 그 맛이 아니...,죄송합니다. 고기나 소스의 변하는 없으나 매일매일 들어오는 재료의 차이가 있을 수 ...,"[('부추전', 0.5247), ('무침', 0.5166), ('특색', 0.331...",고기,,소스,0
2720,포장이 훌륭하네요 국물이 지퍼백에 깔끔하게 와서 너무 연어가 좋았어요 여기저기 찢겨...,안녕하세요~ 상무 초밥 유성점입니다~! 저희 매장을 이용해 주셔서 정말 감사드립니다...,"[('여기저기', 0.529), ('지퍼백', 0.4119), ('실망', 0.34...","상무,상무 초밥",유성점,초밥,0
2721,편의점에서 판매하는 것은 이런 맛이 날까 정도로 얄팍한 밥에 고추냉이가 하나도 없어...,저희는 개인적으로 원하시는 부위를 말씀해 주시면 거기에 맞춰서 적절히 올려 먹는 것...,"[('고추냉이', 0.612), ('편의점', 0.5452), ('부드럽', 0.4...",,,,0
2722,리뷰 작성으로 주신 피스 양념이 많이 되어있는데도 불구하고 비려요 주문한 초밥은 맛...,리뷰 감사합니다 다시 한번 확인하겠습니다,"[('초밥', 0.5357), ('작성', 0.493), ('리뷰', 0.4624)...",,,,0


In [ ]:
df_augmented.to_csv(f"{DATA_PATH}0907_df_neg_augmented.csv",index=False) # 저장

In [ ]:
# aug_reviews = []
# for review in ng_df['corrected_review']:
#     if len(review) > 50:
#         tmp = EDA(review)
#         aug_reviews.extend(tmp)

# aug_reviews

['야채 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 잘 챙겨주세요',
 '야채 단무지 넣지 잘 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 말라고 챙겨주세요',
 '야채 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 잘 챙겨주세요',
 '예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십 년 만에 주문을 해봅니다 양 넉넉했고요 심심하게 먹는 국물은 데쳤는지 딱 간이 안 면은 좀 심심했습니다 대왕오징어는 너무 편인데 거의 아무 맛이 맞았는데 났습니다',
 '예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십 년 만에 주문을 해봅니다 양 넉넉했고요 심심하게 먹는 편인데 국물은 딱 간이 맞았는데 좀 심심했습니다 대왕오징어는 너무 데쳤는지 거의 아무 맛이 안 났습니다',
 '예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십 년 만에 주문을 해봅니다 양 넉넉했고요 심심하게 먹는 편인데 국물은 딱 간이 맞았는데 면은 좀 심심했습니다 대왕오징어는 너무 데쳤는지 거의 아무 맛이 안 났습니다',
 '정말 맛있게 먹었는데 배달 받은 마라탕은 국물도 너무 짜네요 어느 정도냐면 하나 건져먹는데 짠맛이 너무 심해서 도저히 그냥 먹을 수 없을 정도로 짭니다 삼키기가 힘들 정도입니다 ',
 '평소에는 정말 맛있게 먹었는데 오늘 배달 건져먹는데 마라탕은 국물도 적고 그냥 짜네요 어느 정도냐면 재료 하나 받은 정도로 너무 심해서 도저히 너무 먹을 수 없을 짠맛이 짭니다 삼키기가 힘들 정도입니다 ',
 '평소에는 정말 맛있게 먹었는데 오늘 배달 받은 마라탕은 국물도 적고 너무 짜네요 어느 정도냐면 재료 하나 건져먹는데 짠맛이 너무 심해서 도저히 그냥 먹을 수 없을 정도로 짭니다 삼키기가 힘들 정도입니다 ',
 '솔직히 게 못 하나도 랩을 벗기는 순간 눈에 들어온 건 곱창 위에 앉아있는 벌레를 본 순간 입맛이 싹 시켜봤는데 평 하나 보고 첨으로 가시네요 벌레 들어간 음식이 배달될 줄 몰랐네요 별 봤어요 주고 싶

In [ ]:
# pd.DataFrame({'aug_reviews': aug_reviews})

,aug_reviews
0,야채 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 ...
1,야채 단무지 넣지 잘 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 말라...
2,야채 단무지 넣지 말라고 요청드렸는데 주먹밥에 들어거는 김까지 안 보내셨네요 담엔 ...
3,예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십 년 만에 주문을 해봅니다 양 ...
4,예전에는 만다린에서만 주문을 해서 먹었는데 정말 수십 년 만에 주문을 해봅니다 양 ...
...,...
2104,맛은 정말 괜찮은데 기본으로 주는 쌀국수가 양이 너무 적어요 곱빼기에 계란 추가한 ...
2105,맛은 정말 괜찮은데 기본으로 주는 쌀국수가 양이 너무 적어요 곱빼기에 계란 추가한 ...
2106,시원한 국물이 더 많았으면 국물 맛은 시원했을 것 같아요 시원한 국을 시원하게 말이죠
2107,맛은 국물이 더 많았으면 국물 시원한 더 시원했을 것 같아요 시원한 국을 시원하게 ...
